In [14]:
from qiskit import QuantumCircuit
from qiskit.primitives import Estimator
from qiskit.circuit import ParameterVector
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
import numpy as np
from scipy.optimize import minimize

def get_molecule_hamiltonian(driver):
    # Define the electronic structure problem
    es_problem = driver.run()
    
    # Get the Hamiltonian in second quantization
    second_q_op = es_problem.second_q_ops()[0]
    
    # Map to qubit Hamiltonian
    mapper = JordanWignerMapper()
    qubit_hamiltonian = mapper.map(second_q_op)
    
    return qubit_hamiltonian

def ansatz(qubit_hamiltonian):
    # Create a quantum circuit with the required number of qubits
    qc = QuantumCircuit(num_qubits)
    
    # Define a parameter vector for the circuit
    theta = ParameterVector('θ', num_qubits)
    
    # For generic ansatz:
    # Apply parameterized rotations around the Y-axis for each qubit
    for i in range(num_qubits):
        qc.ry(theta[i], i)
    
    # Apply a chain of CNOT gates to entangle adjacent qubits
    for i in range(num_qubits - 1):
        qc.cx(i, i+1)

    # Apply a CNOT between the first and the last qubit
    qc.cx(num_qubits - 1, 0)
    
    return qc, theta

def cost_function(params, hamiltonian, num_qubits):
    qc, theta = ansatz(num_qubits)
    
    # Create a dictionary to assign parameters to the circuit
    param_dict = {theta[i]: params[i] for i in range(num_qubits)}
    
    # Assign parameters to the circuit
    bound_circuit = qc.assign_parameters(param_dict)
    
    # Create an estimator primitive
    estimator = Estimator()
    
    # Calculate the expectation value of the Hamiltonian
    result = estimator.run(bound_circuit, [hamiltonian])
    expectation_value = result.result().values[0]
    
    return expectation_value

def optimization_loop(hamiltonian, num_qubits, optimizer, initial_params):
    # Define the cost function for optimization
    def cost(params):
        return cost_function(params, hamiltonian, num_qubits)
    
    # Perform the optimization
    result = minimize(cost, initial_params, method=optimizer, options={'maxiter': 1000, 'tol': 1e-6})
    
    return result.fun, result.x

# Define the driver for the molecule
driver = PySCFDriver(
    atom="Li 0 0 0; H 0 0 1.5949",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# Get the LiH Hamiltonian
qubit_hamiltonian = get_molecule_hamiltonian(driver)

# The number of qubits can be inferred from the Hamiltonian or qubit operator
num_qubits = qubit_hamiltonian.num_qubits  # Access the number of qubits from the Hamiltonian

# Generate a random initial set of parameters
#initial_params = np.zeros(num_qubits)
initial_params = np.random.uniform(0, 2 * np.pi, num_qubits)
# Define the optimizer
optimizer = 'COBYLA'

# Define the ansatz circuit
def create_ansatz_circuit():
    return ansatz(num_qubits)[0]


# Perform the optimization manually to find the minimum eigenvalue
optimal_value, optimal_params = optimization_loop(qubit_hamiltonian, num_qubits, optimizer, initial_params)
print(f'Optimal energy: {optimal_value} Hartree')

Optimal energy: -7.2089046620378054 Hartree
